In [11]:
from kafka import KafkaProducer
from time import sleep
from json import dumps
from json import loads
import csv

Connect to Kafka

In [12]:
producer = KafkaProducer(bootstrap_servers=['localhost:19092','localhost:19093','localhost:19094'],
                         value_serializer=lambda x: 
                         dumps(x).encode('utf-8'))

Start Streaming

In [13]:
SLEEP = 0

def make_serializable(d):
    serializable_dict = {}
    for key, value in d.items():
        if isinstance(value, (list, dict, str, int, float, bool, type(None))):
            serializable_dict[key] = value
        else:
            serializable_dict[key] = str(value)  # Convert non-serializable value to string
    return serializable_dict

def publish_to_kafka(file_path, producer=producer, chunk_size=1000):
    """ 
    Send each row to kafka mysql topic
    
    Uncomment either the environment code if using .env file or 
    variable code to set directly.
    
    SLEEP variable can be adjusted as desired. 1 is the minimum recommended setting. 
    Setting to 0 will crash the Kafka server unless a more robust EC2 setup is created.
    """
    try:
        with open(file_path, 'r') as file:
            reader = csv.DictReader(file)
            while True:
                chunk = []
                for _ in range(chunk_size):
                    try:
                        chunk.append(next(reader))
                    except StopIteration:
                        break
                if not chunk:
                    break
                for row in chunk:
                    # Convert row to JSON
                    row = make_serializable(row)
                    json_data = row

                    # Publish row to Kafka
                    producer.send('mysql', value=json_data)
                    producer.flush()
                    sleep(SLEEP)
    finally:
        producer.close()

In [14]:
publish_to_kafka("../../../../data/Iowa_Liquor_Sales.csv")

KeyboardInterrupt: 

View Topics

Kafka Docker

1. `docker exec -it kafka1 bash`
1. `cd /usr/bin`
1. `kafka-topics --list --bootstrap-server hostname:9092`
    - Edit the script below for desired topic
1. `kafka-console-consumer --bootstrap-server localhost:9092 --topic mysql --from-beginning`

Manually flush data when done

In [ ]:
producer.flush()